In [92]:
# Manipular grandes volúmenes de datos utilizando Spark SQL para
# resolver un problema

# Machine learning para clasificar
# Python es el lenguaje preferido para la ciencia de datos debido a NumPy,
# Pandas y matplotlib, que son herramientas que facilitan el trabajo con
# matrices y dibujan gráficos y pueden trabajar con grandes matrices de datos
# de manera eficiente. Pero Spark está diseñado para trabajar con una enorme
# cantidad de datos, distribuidos en un clúster.

# Datos del paciente del corazón
# Encontrará adjunto la base de datos.
# Las columnas son:
# 1. Años
# 2. Sexo
# 3. Tipo de dolor torácico (4 valores)
# 4. Presión arterial en reposo
# 5. Colesterol sérico en mg/dl
# 6. Azúcar en sangre en ayunas > 120 mg/dl
# 7. Resultados electrocardiográficos en reposo (valores 0,1,2)
# 8. Frecuencia cardíaca máxima alcanzada
# 9. Angina inducida por el ejercicio
# 10. Oldpeak = depresión del ST inducida por el ejercicio en relación con el reposo
# 11. Pendiente del segmento ST de ejercicio máximo
# 12. Número de vasos principales (0-3) coloreados por fluoroscopia
# 13. Thal: 3 = normal; 6 = defecto fijo; 7 = defecto reversible

# El campo que indica si el paciente tiene un problema cardíaco. Los números
# son los siguientes:

# Un valor de 3 significa que el paciente está sano (normal). Un valor de 6
# significa que se ha solucionado el problema de salud del paciente. Un valor de
# 7 significa que se puede solucionar el problema de salud del paciente.




In [93]:
# importado de librerias
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import IntegerType, FloatType

# Crear una configuración de Spark
conf = SparkConf().setAppName("MyApp").setMaster("local[*]")

sc = SparkContext(conf=conf)

from pyspark.sql import SparkSession
spark = SparkSession(sc)


# Cargar el dataset con esta definicion de columnas y tipos

df = spark.read.csv("heart.dat", header=False, sep=' ')

# set column names
schema = ['Años','Sexo','Tipo_dolor_toracico','Presion_arterial_reposo','Colesterol_serico','Azucar_sangre_ayunas',
            'Resultados_electrocardiográficos_reposo', 'Frecuencia_cardíaca_máxima', 'Angina_inducida_ejercicio',
            'Oldpeak', 'Pendiente_segmento ST','Número_vasos_principales','Thal','Target']
for i in range(len(schema)):
    col = '_c'+str(i) # nombre de la columna
    df = df.withColumn(col, df[col].cast(FloatType())) # convertir a FloatType
    df = df.withColumnRenamed(col, schema[i]) # renombrar columnas
    

   

# Ver el esquema de los datos
df.printSchema()

# Ver los datos
df.show()



root
 |-- Años: float (nullable = true)
 |-- Sexo: float (nullable = true)
 |-- Tipo de dolor torácico: float (nullable = true)
 |-- Presión arterial en reposo: float (nullable = true)
 |-- Colesterol sérico en mg/dl: float (nullable = true)
 |-- Azúcar en sangre en ayunas > 120 mg/dl: float (nullable = true)
 |-- Resultados electrocardiográficos en reposo: float (nullable = true)
 |-- Frecuencia cardíaca máxima alcanzada: float (nullable = true)
 |-- Angina inducida por el ejercicio: float (nullable = true)
 |-- Oldpeak: float (nullable = true)
 |-- Pendiente del segmento ST de ejercicio máximo: float (nullable = true)
 |-- Número de vasos principales: float (nullable = true)
 |-- Thal: float (nullable = true)
 |-- Target: float (nullable = true)

+----+----+----------------------+--------------------------+--------------------------+--------------------------------------+------------------------------------------+------------------------------------+--------------------------------+-

In [88]:
# Entonces, escriba esta función ENFERMO() para marcar 0 como negativo y 1
# como positivo, porque la regresión logística binaria requiere uno de dos
# resultados.

# Importar librerias
from pyspark.sql.functions import udf

# Crear la funcion
def ENFERMO(x):
    if x in(3, 6): # 3 = normal; 6 = problema solucionado
        return 0
    else:
        return 1

# Crear la funcion UDf
udfENFERMO = udf(ENFERMO, IntegerType())

# Aplicar la funcion UDF a la columna target usando el valor de that
df = df.withColumn("Target", udfENFERMO("Thal") )

# Ver los datos
df.show(5)


+----+----+----------------------+--------------------------+--------------------------+--------------------------------------+------------------------------------------+------------------------------------+--------------------------------+-------+---------------------------------------------+---------------------------+----+------+
|Años|Sexo|Tipo de dolor torácico|Presión arterial en reposo|Colesterol sérico en mg/dl|Azúcar en sangre en ayunas > 120 mg/dl|Resultados electrocardiográficos en reposo|Frecuencia cardíaca máxima alcanzada|Angina inducida por el ejercicio|Oldpeak|Pendiente del segmento ST de ejercicio máximo|Número de vasos principales|Thal|Target|
+----+----+----------------------+--------------------------+--------------------------+--------------------------------------+------------------------------------------+------------------------------------+--------------------------------+-------+---------------------------------------------+---------------------------+----+---

In [94]:
# También debe crear el dataframe de Spark raw_data usando la operación
# transform() y seleccionando solo la columna de características.

# Importar librerias
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# set columns except the col Target
columns = df.columns
columns.remove('Target')


# Crear el vector assembler
assembler = VectorAssembler(inputCols=columns, outputCol='features')

# Transformar los datos
raw_data = assembler.transform(df).select("features")

# Ver los datos
raw_data.show(5)


+--------------------+------+
|            features|Target|
+--------------------+------+
|[70.0,1.0,4.0,130...|   2.0|
|[67.0,0.0,3.0,115...|   1.0|
|[57.0,1.0,2.0,124...|   2.0|
|[64.0,1.0,4.0,128...|   1.0|
|[74.0,0.0,2.0,120...|   1.0|
+--------------------+------+
only showing top 5 rows



In [91]:
#cerrar la sesion de spark
spark.stop()


In [ ]:
# - SQL Spark

# SQL Spark es un módulo de Spark para el procesamiento de datos estructurados.
# Proporciona una API para manipular datos estructurados utilizando el lenguaje
# SQL estándar. Puede ejecutar consultas SQL escritas en el lenguaje SQL estándar
# utilizando Spark SQL. Spark SQL proporciona una API para manipular datos
# estructurados utilizando el lenguaje SQL estándar. Puede ejecutar consultas SQL
# escritas en el lenguaje SQL estándar utilizando Spark SQL. Spark SQL proporciona
# una API para manipular datos estructurados utilizando el lenguaje SQL estándar.
# Puede ejecutar consultas SQL escritas en el lenguaje SQL estándar utilizando
# Spark SQL.

# - Formatos de Archivo
# Spark SQL proporciona una API para leer archivos de texto, JSON, CSV, parquet,
# ORC y muchos otros formatos de archivo. Puede leer datos de estos formatos y
# crear un DataFrame a partir de ellos. También puede escribir datos en estos
# formatos.

